```bash
cd `dirname $0`/.. && wk_dir=`pwd` && cd -
source ${wk_dir}/util/env.sh
 
module_name=`basename ${wk_dir}`
job_name=`basename $0 .sh`
 
if [ $# -ne 3 ] ; then
  echo "usage: $0 dt conf_dir platform_conf_dir"
  exit 255
fi
platform_conf_dir="$3"
source ${platform_conf_dir}/platform.conf
###############################################################################
 
echo "date:" ${d1}
echo "arg platform:" ${platform}
 
input_file="${hive_dir}/${table_user_attributes_json}/platform=${platform}/dt=${d1}"
output_file="${hive_dir}/tmp/tmp_redis_import/platform=${platform}/dt=${d1}"
reducer="reducer_import_redis.py"
reducer_depend="constant.py"
archive="/user/ad_dm/packages/simplejson.tar.gz#simplejson"
localpath="${wk_dir}/log/redis_log/$platform/$d1"
mkdir -p $localpath
 
cmd="${HADOOP} jar ${hadoop_jar}
     -D mapred.map.tasks=100
     -D mapred.reduce.tasks=100
     -D stream.map.input.ignoreKey=true
     -inputformat com.hadoop.mapred.DeprecatedLzoTextInputFormat
     -input ${input_file}
     -output ${output_file}
     -file ${reducer}
     -file ${reducer_depend}
     -mapper cat
     -reducer ${reducer}
     -cacheArchive ${archive}"
echo ${output_file}
${HADOOP} fs -rmr ${output_file}
echo $cmd
$cmd
```